# JOBS WEB SCRAPPER

## TO DO :

- how to get pubilshing data
- for a given set of words (ex : PowerBI), analyze frequency, use in given companies, evolution of use, ...

In [ ]:
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException

import bs4
import urllib


import random

## LINKEDIN

In [ ]:
driver = webdriver.Chrome()

email = ""
password = ""

# Go to linkedin and login
driver.get('https://www.linkedin.com/login')
time.sleep(3)
driver.find_element_by_id('username').send_keys(email)
driver.find_element_by_id('password').send_keys(password)
driver.find_element_by_id('password').send_keys(Keys.RETURN)
time.sleep(5)

driver.maximize_window()

driver.get("https://www.linkedin.com/jobs/")
time.sleep(5)

#ActionChains(driver).click(driver.find_element_by_xpath('//*[@id="ember121"]/div/div/p[1]')).perform()

In [ ]:
dic_annonces = {'title':[],'company':[],'place':[],'n_appliants':[],'content':[],
               'type':[], 'ppl_domain':[], 'date':[]}

# scrolls a single page:
for n_page in range(30):
    
    # get a list of all the listings elements's in the side bar
    list_items = driver.find_elements_by_class_name("occludable-update")
    
    for each_job in list_items:

        # executes JavaScript to scroll the div into view
        driver.execute_script("arguments[0].scrollIntoView();", each_job)
        each_job.click()
        time.sleep(random.randint(5,10))

        # get info:

        title_temp = each_job.text.split('\n')[0]
        company_temp = each_job.text.split('\n')[1]
        place_temp = each_job.text.split('\n')[2]
        
        if len(each_job.text.split('\n')) > 6 :
               n_appliants_temp = each_job.text.split('\n')[6]
        else :
               n_appliants_temp = "unknown"
        
        try:
            content_temp = "Content : " + driver.find_element_by_id("job-details").text
        except NoSuchElementException:
            content_temp = "NA"
        
        try:
            type_temp = driver.find_element_by_xpath('/html/body/div[6]/div[3]/div[3]/div[2]/div/section[2]/div/div/div[1]/div/div[1]/div/div[2]/div[2]/div[1]/span').text
        except NoSuchElementException:
            type_temp = "NA"
        
        try:
            date_temp = driver.find_element_by_xpath('/html/body/div[6]/div[3]/div[3]/div[2]/div/section[2]/div/div/div[1]/div/div[1]/div/div[2]/div[1]/span[2]/span[1]').text
        except NoSuchElementException:
            date_temp = "NA"
        
        try:
            ppl_domain_temp = driver.find_element_by_xpath('/html/body/div[6]/div[3]/div[3]/div[2]/div/section[2]/div/div/div[1]/div/div[1]/div/div[2]/div[2]/div[2]/span').text
        except NoSuchElementException:
            ppl_domain_temp = "NA"
        
        dic_annonces['title'].append(title_temp)
        dic_annonces['company'].append(company_temp)
        dic_annonces['place'].append(place_temp)
        dic_annonces['n_appliants'].append(n_appliants_temp)
        dic_annonces['content'].append(content_temp)
        dic_annonces['ppl_domain'].append(ppl_domain_temp)
        dic_annonces['date'].append(date_temp)
        dic_annonces['type'].append(type_temp)

    time.sleep(10)
    
    first_url = driver.current_url
    
    driver.get(driver.current_url+'&start='+str(len(dic_annonces['title'])))

In [ ]:
RTE = pd.DataFrame.from_dict(dic_annonces)
display(RTE.head())
RTE.info()

In [ ]:
def prep_dataset(df) :
    
    df = df.applymap(lambda s:s.upper() if type(s) == str else s)
    df['place'] = df['place'].str.split(',', expand = True)[0]
    df['n_appliants'] = df['n_appliants'].str.split(' ', expand = True)[0]
    df['size_company'] = df['ppl_domain'].str.split('·', expand = True)[0]
    #df['domain_company'] = df['ppl_domain'].str.split('·', expand = True)[1]
    df = df.drop(['ppl_domain','n_appliants'], axis = 1)
    df['content'] = df['content'].str.replace('\n','')
    
    #print(df.head())
    
    return df

In [ ]:
RTE = prep_dataset(RTE)

In [ ]:
RTE.info()

In [ ]:
RTE.head(50)

## INDEED

## GLASSDOOR

## CADREMPLOI